In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.models import load_model
from kerastuner import RandomSearch

/var/folders/_0/n_t3__qd6k15bv63l5bktb8m0000gn/T/ipykernel_53839/3515859741.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
num_classes = 2

# Image Data Generators for augmenting and rescaling images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
# Load training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=32,
    class_mode='categorical'  # Categorical for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [5]:
number_of_class = 2

In [6]:
model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

model.trainable = False

x = model.output
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)

# Add fully connected layers with Dropout for regularization
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(number_of_class, activation='softmax')(x)

In [7]:
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
vgg_16_model = Model(
    inputs=model.input,
    outputs=predictions
)

In [9]:
# Compile the model
vgg_16_model.compile(
    optimizer=Adam(learning_rate = 0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# Display the model summary
vgg_16_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 512)      │             

 Total params: 20,620,354 (78.66 MB)

 Trainable params: 5,903,618 (22.52 MB)

 Non-trainable params: 14,716,736 (56.14 MB)

### Model Training

In [11]:
number_of_epochs = 30

In [12]:
# Train the model
history = vgg_16_model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = number_of_epochs  # Adjust the number of epochs based on your dataset size
)

Epoch 1/30


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.4657 - loss: 1.1806 - val_accuracy: 0.6429 - val_loss: 0.6868
Epoch 2/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5000 - loss: 1.0969

2024-11-06 23:40:57.336687: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 524ms/step - accuracy: 0.5000 - loss: 1.0969 - val_accuracy: 0.6071 - val_loss: 0.6864
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5511 - loss: 0.9970 - val_accuracy: 0.5714 - val_loss: 0.6892
Epoch 4/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5000 - loss: 1.0180

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 527ms/step - accuracy: 0.5000 - loss: 1.0180 - val_accuracy: 0.6429 - val_loss: 0.6850
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.4884 - loss: 1.1030 - val_accuracy: 0.5714 - val_loss: 0.6799
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 514ms/step - accuracy: 1.0000 - loss: 0.1840 - val_accuracy: 0.5357 - val_loss: 0.6794
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5242 - loss: 0.9666 - val_accuracy: 0.5000 - val_loss: 0.6884
Epoch 8/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5312 - loss: 1.1420

2024-11-06 23:42:00.925383: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 533ms/step - accuracy: 0.5312 - loss: 1.1420 - val_accuracy: 0.6071 - val_loss: 0.6781
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5961 - loss: 0.8353 - val_accuracy: 0.6071 - val_loss: 0.6718
Epoch 10/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 521ms/step - accuracy: 0.6875 - loss: 0.7202 - val_accuracy: 0.5714 - val_loss: 0.6759
Epoch 11/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5904 - loss: 0.8420 - val_accuracy: 0.5714 - val_loss: 0.6743
Epoch 12/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 517ms/step - accuracy: 0.6250 - loss: 0.8286 - val_accuracy: 0.6786 - val_loss: 0.6702
Epoch 13/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6357 - loss: 0.7418 - val_accuracy: 0.6429 - val_loss: 0.6708
Epoch 14/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 518ms/step - accuracy: 0.7188 - loss: 0.5628 - val_accuracy: 0.7143 - val_loss: 0.6618
Epoch 15/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.6496 - loss: 0.7798 - val_accuracy: 0.6786 - val_loss: 0.6610
Epoch

2024-11-06 23:43:26.140660: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 547ms/step - accuracy: 1.0000 - loss: 0.0782 - val_accuracy: 0.6071 - val_loss: 0.6704
Epoch 17/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5902 - loss: 0.7285 - val_accuracy: 0.7500 - val_loss: 0.6637
Epoch 18/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 515ms/step - accuracy: 0.7188 - loss: 0.5192 - val_accuracy: 0.6429 - val_loss: 0.6610
Epoch 19/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.6618 - loss: 0.7498 - val_accuracy: 0.7500 - val_loss: 0.6548
Epoch 20/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 516ms/step - accuracy: 0.5625 - loss: 0.7500 - val_accuracy: 0.7500 - val_loss: 0.6557
Epoch 21/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.7200 - loss: 0.6310 - val_accuracy: 0.7143 - val_loss: 0.6545
Epoch 22/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 555ms/step - accuracy: 0.7812 - loss: 0.4321 - val_accuracy: 0.7143 - val_loss: 0.6471
Epoch 23/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.7298 - loss: 0.6036 - val_accuracy: 0.7143 - val_loss: 0.6438
Epoc

In [13]:
# Optionally, unfreeze some layers in the base model and fine-tune
model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
vgg_16_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
# Fine-tune the model
history_finetune = vgg_16_model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = 10  # Fine-tune for a few more epochs
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 41s 6s/step - accuracy: 0.7348 - loss: 0.5086 - val_accuracy: 0.6786 - val_loss: 0.5973
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 31s 6s/step - accuracy: 0.6562 - loss: 0.7421

2024-11-06 23:46:44.651446: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 512ms/step - accuracy: 0.6562 - loss: 0.7421 - val_accuracy: 0.6786 - val_loss: 0.6059
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 45s 7s/step - accuracy: 0.7775 - loss: 0.4908 - val_accuracy: 0.7500 - val_loss: 0.6082
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 557ms/step - accuracy: 0.0000e+00 - loss: 1.4227 - val_accuracy: 0.7500 - val_loss: 0.6124
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 40s 6s/step - accuracy: 0.7425 - loss: 0.5180 - val_accuracy: 0.6429 - val_loss: 0.6539
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 522ms/step - accuracy: 0.7812 - loss: 0.4989 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.8097 - loss: 0.4367 - val_accuracy: 0.5357 - val_loss: 0.6861
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 521ms/step - accuracy: 0.6875 - loss: 0.7089 - val_accuracy: 0.5357 - val_loss: 0.6775
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.7475 - loss: 0.4492 - val_accuracy: 0.5357 - val_loss: 0.6790
Epoch 1

In [15]:
# Save the model, optional
vgg_16_model.save(f'Pth_Files/VGG16_{number_of_epochs}.keras')

## Model Testing

In [16]:
def load_image_test(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array /= 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)
    predicted_probability = np.max(predictions) * 100  # Get the probability of the predicted class


    # If you know the class labels (e.g., from a class_indices mapping when using a data generator)
    class_labels = {0: 'Class1', 1: 'Class2', 2: 'Class3'}  # Update with your actual labels
    predicted_label = class_labels[predicted_class_index[0]]

    # Print the prediction results
    print(f"Predicted class index: {predicted_class_index[0]}")
    print(f"Predicted class label: {predicted_label}")
    print(f"Prediction confidence: {predicted_probability:.2f}%")

In [17]:
# Load your trained model
model = load_model(f'Pth_Files/VGG16_{number_of_epochs}.keras')

In [18]:
test_img_path = test_dir + '/Agaricus_abruptibulbus/Agaricus_abruptibulbus_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Predicted class index: 0
Predicted class label: Class1
Prediction confidence: 53.24%


In [19]:
test_img_path = test_dir + '/Amanita_excelsa/Amanita_excelsa_1.jpg'
img_array = load_image_test(test_img_path, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Predicted class index: 1
Predicted class label: Class2
Prediction confidence: 61.70%
